In [5]:
import pandas as pd
import numpy as np

from sklearn.metrics import f1_score as f1
from sklearn.pipeline import Pipeline

import autosklearn.classification
import warnings
warnings.simplefilter('ignore')

/home/peiwang/anaconda3/lib/python3.7/site-packages/pyparsing.py:3168: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


In [6]:
for i in range(7, 9):
    for t in [3600]:
        train_path = './02-feature-extract-ori/features/ft_' + str(i) + '_train.csv'
        test_path = './02-feature-extract-ori/features/ft_' + str(i) + '_test.csv'
        
        df_train = pd.read_csv(train_path)
        df_test = pd.read_csv(test_path)
        
        X_train = np.array(df_train.drop(['label'], axis=1).values.tolist())
        y_train = np.array(df_train['label'].values.tolist())
        X_test = np.array(df_test.drop(['label'], axis=1).values.tolist())
        y_test = np.array(df_test['label'].values.tolist())
        print("data_" + str(i), "training data size:", len(y_train), "test data size:", len(y_test))
        
        automl = autosklearn.classification.AutoSklearnClassifier(
            include_estimators=['random_forest'], 
            time_left_for_this_task=t, 
            initial_configurations_via_metalearning=0, 
            ensemble_nbest=1, ensemble_size=1, 
            resampling_strategy='holdout',
            resampling_strategy_arguments={'train_size':0.75}
        )
        automl.fit(X_train, y_train, metric=autosklearn.metrics.f1)
        print("random-forest-selected, training fscore:", automl.score(X_train, y_train),  ", test fscore: ", automl.score(X_test, y_test))
        print(automl.show_models())
        print(automl.sprint_statistics())

data_7 training data size: 8193 test data size: 2049
[WARNING] [2020-10-06 15:02:02,540:EnsembleBuilder(1):a619ef2c1fe74f7db94dc7cc716bf72b] No models better than random - using Dummy Score!
[WARNING] [2020-10-06 15:02:02,550:EnsembleBuilder(1):a619ef2c1fe74f7db94dc7cc716bf72b] No models better than random - using Dummy Score!
[WARNING] [2020-10-06 15:02:04,554:EnsembleBuilder(1):a619ef2c1fe74f7db94dc7cc716bf72b] No models better than random - using Dummy Score!
random-forest-selected, training fscore: 0.9052102950408034 , test fscore:  0.7938931297709924
[(1.000000, SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'categorical_encoding:__choice__': 'one_hot_encoding', 'classifier:__choice__': 'random_forest', 'imputation:strategy': 'most_frequent', 'preprocessor:__choice__': 'select_percentile_classification', 'rescaling:__choice__': 'robust_scaler', 'categorical_encoding:one_hot_encoding:use_minimum_fraction': 'False', 'classifier:random_forest:bootstrap': 'True', 'cl

In [2]:
# validate if the data processing steps contributes to improvment (manual)

X_train_list = []
y_train_list = []
X_test_list = []
y_test_list = []

for i in range(1, 9):
    train_path = './02-feature-extract-ori/features/ft_' + str(i) + '_train.csv'
    test_path = './02-feature-extract-ori/features/ft_' + str(i) + '_test.csv'
    
    df_train = pd.read_csv(train_path)
    df_test = pd.read_csv(test_path)
        
    X_train = np.array(df_train.drop(['label'], axis=1).values.tolist())
    y_train = np.array(df_train['label'].values.tolist())
    X_test = np.array(df_test.drop(['label'], axis=1).values.tolist())
    y_test = np.array(df_test['label'].values.tolist())
    print("data_" + str(i), "training data size:", len(y_train), "test data size:", len(y_test))
    
    X_train_list.append(X_train)
    y_train_list.append(y_train)
    X_test_list.append(X_test)
    y_test_list.append(y_test)

data_1 training data size: 359 test data size: 91
data_2 training data size: 757 test data size: 189
data_3 training data size: 430 test data size: 109
data_4 training data size: 9890 test data size: 2473
data_5 training data size: 22965 test data size: 5742
data_6 training data size: 9167 test data size: 2293
data_7 training data size: 8193 test data size: 2049
data_8 training data size: 7659 test data size: 1916


In [ ]:
# exp for dataset 1

# random-forest-selected, training fscore: 0.9622641509433962 , test fscore:  0.8125000000000001
# [(1.000000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'categorical_encoding:__choice__': 'one_hot_encoding', 'classifier:__choice__': 'random_forest', 'imputation:strategy': 'median', 'preprocessor:__choice__': 'select_rates', 'rescaling:__choice__': 'robust_scaler', 'categorical_encoding:one_hot_encoding:use_minimum_fraction': 'True', 'classifier:random_forest:bootstrap': 'False', 'classifier:random_forest:criterion': 'gini', 'classifier:random_forest:max_depth': 'None', 'classifier:random_forest:max_features': 0.9353269112541961, 'classifier:random_forest:max_leaf_nodes': 'None', 'classifier:random_forest:min_impurity_decrease': 0.0, 'classifier:random_forest:min_samples_leaf': 3, 'classifier:random_forest:min_samples_split': 15, 'classifier:random_forest:min_weight_fraction_leaf': 0.0, 'classifier:random_forest:n_estimators': 100, 'preprocessor:select_rates:alpha': 0.3389866291265189, 'preprocessor:select_rates:mode': 'fdr', 'preprocessor:select_rates:score_func': 'chi2', 'rescaling:robust_scaler:q_max': 0.75, 'rescaling:robust_scaler:q_min': 0.25, 'categorical_encoding:one_hot_encoding:minimum_fraction': 0.14640208844635372},
# dataset_properties={
#   'task': 1,
#   'sparse': False,
#   'multilabel': False,
#   'multiclass': False,
#   'target_type': 'classification',
#   'signed': False})),
# ]

X_train = X_train_list[1]
y_train = y_train_list[1]
X_test = X_test_list[1]
y_test = y_test_list[1]

pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])

In [ ]:
from autosklearn.pipeline.classification import SimpleClassificationPipeline
piepline = SimpleClassificationPipeline({'balancing:strategy': 'none', 
                              'categorical_encoding:__choice__': 'one_hot_encoding', 
                              'classifier:__choice__': 'random_forest', 
                              'imputation:strategy': 'median', 
                              'preprocessor:__choice__': 'select_rates', 
                              'rescaling:__choice__': 'robust_scaler', 
                              'categorical_encoding:one_hot_encoding:use_minimum_fraction': 'True', 
                              'classifier:random_forest:bootstrap': 'False', 
                              'classifier:random_forest:criterion': 'gini', 
                              'classifier:random_forest:max_depth': 'None', 
                              'classifier:random_forest:max_features': 0.9353269112541961, 
                              'classifier:random_forest:max_leaf_nodes': 'None', 
                              'classifier:random_forest:min_impurity_decrease': 0.0, 
                              'classifier:random_forest:min_samples_leaf': 3, 
                              'classifier:random_forest:min_samples_split': 15, 
                              'classifier:random_forest:min_weight_fraction_leaf': 0.0, 
                              'classifier:random_forest:n_estimators': 100, 
                              'preprocessor:select_rates:alpha': 0.3389866291265189, 
                              'preprocessor:select_rates:mode': 'fdr', 
                              'preprocessor:select_rates:score_func': 'chi2', 
                              'rescaling:robust_scaler:q_max': 0.75, 
                              'rescaling:robust_scaler:q_min': 0.25, 
                              'categorical_encoding:one_hot_encoding:minimum_fraction': 0.14640208844635372},